In [ ]:
import hist
import numpy as np
import matplotlib.pyplot as plt

# Make a random number generator
rng = np.random.default_rng()

# Classic histograms in 1D

Let's start with the basics. We will create a histogram the classic way, using a simple, random dataset.

## Simple introduction

Let's make a 1d dataset to run on.

In [ ]:
data1 = rng.normal(3.5, 2.5, size=1_000_000)

This is a random, gaussian dataset. If you wanted to visualize it, you would probably would _NOT_ do this (plotting only the first 1,000 points just to keep SVG/PDF backends from bogging down):


In [ ]:
plt.plot(data1[:1_000], '.');

Instead, you would usually look at your data using a histogram like this:

In [ ]:
plt.hist(data1, bins='auto');

Now you can infer much more about the distribution; you can see the shape and the width directly from this image.
This is _one_ of the uses of a histogram, often the one you are most familar with; it's a vizualization tool. But it's much, much more too.

If you wanted to actually convert this to a histogram yourself, you can use NumPy's built in function:

In [ ]:
np.histogram(data1, range=(-5, 10), bins=10)

The return value here is two NumPy arrays, one with bin contents (frequencies), and the other with bin edges. You are responsible for keeping these in sync if you manipulate further.

We can smoothy transition to using the hist library by using the `hist.numpy` adaptor.

In [ ]:
hist.numpy.histogram(data1, range=(-5, 10), bins=10)

Is there a reason we might want to do this (other than to get to the remainder of the lesson)? Let's check performance:

In [ ]:
%%timeit
_,_ = np.histogram(data1, bins=100)

In [ ]:
%%timeit
_,_ = hist.numpy.histogram(data1, bins=100, threads=2)

Hist is backed by the boost-histogram library, providing compiled performance based on the C++ Boost.Histogram. We'll see even more impressive speedups later; NumPy optimizes for regular binning in 1D, but not elsewhere.

## The histogram object

Now, let's see what a Histogram object looks like.

In [ ]:
h = hist.numpy.histogram(data1, bins=100, histogram=hist.Hist)
h

In [ ]:
h.plot();

This is now an _object_, rather than a tuple of NumPy arrays.

Now let's graduate from the NumPy interface, and make a Histogram directly without the adaptor:

In [ ]:
h = hist.Hist(hist.axis.Regular(300, -5, 10, name="x"))
h.fill(data1)

There is something to notice here; the fill method mutates the histogram object (it does return itself, so you can chain this functional style if you want). This means that you can fill the histogram multiple times, it's not one-and-done like NumPy!

You can slice it:

In [ ]:
h[20:40]

Notice how the bin information is propogated with the histogram.


If you want to slice in data coordinates, just use data coordinates and add a "j" suffix. The same slice as above:

In [ ]:
h[-4j:-3j]

You can even mix location and bin number access. You can also rebin by placing a `j` suffixed number in the final slice slot, which we call the "action". This looks quite a bit like a step, but instead of every Nth bins, you are merging N adjacent bins.

In [ ]:
h[::10j]

Let's look at some information about the histogram. First, how about the sum?

In [ ]:
h.sum()

That's not the total number of entries we put in. Why not?

---
<details><summary>Click to show answer</summary>

Answer: they were outside the range of the Axis. Some of them were too large, some were too low. There are special (and optional) bins for these two cases, called the underflow and the overflow. We can include them in the sum:
    
```python
h.sum(flow=True)
```
    
Later these will enable some powerful things like lossless projections. Most of the time, you can ignore they are there.
    
</details>

Let's plot this by hand instead of calling `.plot()`:

In [ ]:
plt.bar(h.axes[0].centers, h.values(), width=h.axes[0].widths);

Note: You can leave off the `.values()` if you want to - histograms conform to the Python buffer protocol and directly expose their `.view()` memory. There will be a lot more on this later. The above will work with any storage, making it more correct in this case.

You can begin to see how, while we are currently discussing 1D histograms, ND histograms have exactly the same interface, just with more axes.

Aside: here's step. It's quite ugly for us out of the box in matplotlib, just like it is for numpy. In Matplotlib 3.4, we have added a `plt.stairs` function which makes plotting beautiful pre-binned histograms much simpler.

In [ ]:
plt.step(h.axes[0].edges[:-1], h, where='post');

This is not _quite_ right, the edges are inconsistent (as you might gather from having to leave one edge off above) but `plt.stairs` fixes this; polishing it off manualy is quite a bit of effort.


In [ ]:
plt.stairs(h, h.axes[0].edges);

Now that's better!

## Properties

There are some fantastic properties of Histograms that make analysis simple and easy. Let's look at axis breifly, using a rebinned histogram to keep the display short and sweet:

In [ ]:
hs = h[::30j]

In [ ]:
hs.axes[0].centers

In [ ]:
hs.axes[0].edges

In [ ]:
hs.axes[0].widths

In [ ]:
hs.axes[0].name

The PlottableHistogram Protocol has severl useful methods:

In [ ]:
hs.values()

In [ ]:
hs.variances()  # may be None for an unweighted histogram, depending on the fill

In [ ]:
hs.counts()  # more interesting later

And, of course, we can directly look at the underlying memory, which is pretty simple in this case:

In [ ]:
hs.view()

## Making a density histogram

Let's try to make a density histogram like NumPy's/matplotlib's. This is how you would do with in those packages:

In [ ]:
# Interesting edges
bins = [-10, -7, -4, -3, -2, -1, -.75, -.5, -.25, 0, .25, .5, .75, 1, 2, 3, 4, 7, 10]

# Make a density array and edges array
d7, e7 = np.histogram(data1 - 3.5, bins=bins, density=True)

# Plot (recompute histogram)
plt.hist(data1 - 3.5, bins=bins, density=True);

Yes, it's ugly. Don't judge.

Yes, I'm aware there's a `.density()` method in Hist and a density keyword in plot:

In [ ]:
hd = hist.Hist(hist.axis.Variable(bins))
hd.fill(data1 - 3.5)
hd.plot(density=True);

But that's not fun and instructive, so let's try making out of basic building blocks.

In [ ]:
widths = hd.axes.widths
area = np.prod(widths, axis=0)

area

Yes, that does not need to be so complicated for 1D, but it's general; this exact procedure will also support ND histograms!

In [ ]:
factor = np.sum(hd.values())
density = hd.values() / (factor * area)

In [ ]:
plt.bar(hd.axes[0].centers, density, width=hd.axes[0].widths);

And there it is! Each of these different properties worked together to provide a basis for computing the density. If we wanted to compute other quanities, these same building blocks would be easy to use for those, too.

In [ ]:
density, hd.density()

Let's break up and try worksheet one!